# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686568


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:24,  2.92s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:11,  2.54s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:50,  1.86s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:37,  1.43s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:13<00:00,  5.07it/s]

Rendering models:  94%|█████████▎| 29/31 [00:13<00:00,  3.99it/s]

Rendering models:  97%|█████████▋| 30/31 [00:13<00:00,  3.75it/s]

Rendering models: 100%|██████████| 31/31 [00:14<00:00,  3.65it/s]

not-logged-in-562d9daec75faadff5d4    0.000261
not-logged-in-20a275b53abec8db9f41    0.019980
not-logged-in-20a275b53abec8db9f41    0.045378
Slayz                                 0.042448
shirogane                             0.000235
not-logged-in-a9247261537fa1da33fe    0.134794
not-logged-in-a9247261537fa1da33fe    0.011788
CThomas                               0.016275
cschwefl                              0.000436
Dynamatt                              0.000242
xil31                                 0.000688
tbel765                               0.012562
casbee                                0.002185
Daniel.brooker2                       0.006504
sn346808                              0.026222
Finn.Ralston                          0.001530
not-logged-in-ffc5c906399855f5ef0f    0.000328
KrisKeogh                             0.058320
superphoton                           0.000180
not-logged-in-14d09f70263c1fe15cc3    0.001014
not-logged-in-c94e8595af40a920d279    0.000987
dabiggreenbus

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())